#### Importing and preprocessing the data

In [27]:
import pandas as pd
from sklearn.model_selection import train_test_split
from pyfume.Clustering import Clusterer
from pyfume.EstimateAntecendentSet import AntecedentEstimator
from pyfume.EstimateConsequentParameters import ConsequentEstimator
from pyfume.SimpfulModelBuilder import SugenoFISBuilder
from pyfume.Tester import SugenoFISTester
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, cohen_kappa_score
from numpy import clip, column_stack, argmax
from sklearn.neural_network import MLPClassifier

data = pd.read_csv('wbco.csv', header=None)
indexes = ['LThick', 'UCellSize', 'UCellShape', 'MAdhesion', 'SECS', 'BlandC', 'NormNuc', 'Mitoses', 'Unknown', 'Target']
data.columns = indexes

indexes = indexes[:-1]

In [28]:
data.head()

,LThick,UCellSize,UCellShape,MAdhesion,SECS,BlandC,NormNuc,Mitoses,Unknown,Target
0,5,1,1,1,2,1,3,1,1,0
1,5,4,4,5,7,10,3,2,1,0
2,3,1,1,1,2,2,3,1,1,0
3,6,8,8,1,3,4,3,7,1,0
4,4,1,1,3,2,1,3,1,1,0


In [29]:
data.describe()

,LThick,UCellSize,UCellShape,MAdhesion,SECS,NormNuc,Mitoses,Unknown,Target
count,699.000000,699.000000,699.000000,699.000000,699.000000,699.000000,699.000000,699.000000,699.000000
mean,4.417740,3.134478,3.207439,2.806867,3.216023,3.437768,2.866953,1.589413,0.344778
std,2.815741,3.051459,2.971913,2.855379,2.214300,2.438364,3.053634,1.715078,0.475636
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
25%,2.000000,1.000000,1.000000,1.000000,2.000000,2.000000,1.000000,1.000000,0.000000
50%,4.000000,1.000000,1.000000,1.000000,2.000000,3.000000,1.000000,1.000000,0.000000
75%,6.000000,5.000000,5.000000,4.000000,4.000000,5.000000,4.000000,1.000000,1.000000
max,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,1.000000


In [30]:
data.isnull().sum()

LThick        0
UCellSize     0
UCellShape    0
MAdhesion     0
SECS          0
BlandC        0
NormNuc       0
Mitoses       0
Unknown       0
Target        0
dtype: int64

In [31]:
data.dtypes == int

LThick         True
UCellSize      True
UCellShape     True
MAdhesion      True
SECS           True
BlandC        False
NormNuc        True
Mitoses        True
Unknown        True
Target         True
dtype: bool

A variável 'BlandC' está atualmente definida como string pois possui algumas entradas com '?'. Como resolver este problema? O número de entradas sem valor é significativo?

In [32]:
na_perc = (data.BlandC[data.BlandC == '?']).count() / data.BlandC.count() * 100
print(str(na_perc) + '% percent of entries are null.')

2.28898426323319% percent of entries are null.


Selected aproach: fill missing values with the column's average.

In [33]:
avg = data.BlandC[data.BlandC != '?'].astype(float).mean()
data.loc[data.BlandC == '?', 'BlandC'] = avg
data.BlandC = data.BlandC.astype(int)

Feature Selection

In [34]:
correlations = data.drop('Target', axis = 1).corrwith(data['Target']) 
print(correlations)

LThick        0.716001
UCellSize     0.817904
UCellShape    0.818934
MAdhesion     0.696800
SECS          0.682785
BlandC        0.817442
NormNuc       0.756616
Mitoses       0.712244
Unknown       0.423170
dtype: float64


In [35]:
data = data.drop('Unknown', axis = 1)

Creating a train-test split

In [36]:
data_x = data.drop('Target', axis = 1)
data_y = data.Target

x_train, x_test, y_train, y_test = train_test_split(data_x, data_y, test_size = 0.2, random_state=21)

Formating the data to numpy

In [37]:
x_train = x_train.to_numpy()
x_test = x_test.to_numpy()
y_train = y_train.array
y_test = y_test.array

#### Building the model

Neural network model

In [38]:
def trainMLP(HLS):
    regr = MLPClassifier(hidden_layer_sizes=HLS,random_state=42, max_iter=500)
    regr.fit(x_train, y_train)

    # %% Get model predictions
    y_pred = regr.predict(x_test)

    # %% Compute classification metrics
    print('')
    print('Architecture: {list}'.format(list = HLS))
    acc_score = accuracy_score(y_test, y_pred)
    print("Accuracy: {:.3f}".format(acc_score))
    rec_score = recall_score(y_test, y_pred)
    print("Recall: {:.3f}".format(rec_score))
    prec_score = precision_score(y_test, y_pred)
    print("Precision Score: {:.3f}".format(prec_score))
    F1_score = f1_score(y_test, y_pred)
    print("F1-Score: {:.3f}".format(F1_score))
    kappa = cohen_kappa_score(y_test, y_pred)
    print("Kappa Score: {:.3f}".format(kappa))


In [39]:
Architectures = [[20],[30],[30, 30],[50]]

for i in Architectures: trainMLP(i)


Architecture: [20]
Accuracy: 0.943
Recall: 0.887
Precision Score: 0.959
F1-Score: 0.922
Kappa Score: 0.877


/Users/alexandregoncalves/miniconda3/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(



Architecture: [30]
Accuracy: 0.936
Recall: 0.868
Precision Score: 0.958
F1-Score: 0.911
Kappa Score: 0.861

Architecture: [30, 30]
Accuracy: 0.936
Recall: 0.868
Precision Score: 0.958
F1-Score: 0.911
Kappa Score: 0.861

Architecture: [50]
Accuracy: 0.964
Recall: 0.943
Precision Score: 0.962
F1-Score: 0.952
Kappa Score: 0.924


/Users/alexandregoncalves/miniconda3/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
